### Sentiment anlayis for the Reviews Using VENDER

 

<b> Importing needed Libaries

In [ ]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
import string
from nltk.stem import PorterStemmer


<b>Creating Data Frame with the csv file

In [ ]:

filePath = '../dataset/amazon_review_updated_label.txt'
df = pd.read_csv(filePath, delimiter='\t')
df.head()

<b>Using VADER to analayze the sentiment of the reviews


In [ ]:
df = df.drop(['DOC_ID', 'RATING', 'VERIFIED_PURCHASE', 'PRODUCT_CATEGORY', 'PRODUCT_ID', 'PRODUCT_TITLE', 'REVIEW_TITLE'], axis="columns")
df

In [ ]:
# Getting a compund score
def sentimental_compound(sentence):
    analyzer = SentimentIntensityAnalyzer()
    df_sentiment = analyzer.polarity_scores(sentence)
    score=df_sentiment['compound']
    return score

# VADER predicting Positive negative
def sentimental_score(sentence):
    analyzer = SentimentIntensityAnalyzer()
    df_sentiment = analyzer.polarity_scores(sentence)
    score=df_sentiment['compound']
    if score >= 0:
        return 'pos'
    else:
        return 'neg'

### Importing SVM classification

In [ ]:
#!pip3 install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [ ]:
# # selecting 1000 rows to avoid crashing the browser
# df_selected_rows = df.head(1000)
# #df.insert(1, "lab",0)
# df_selected_rows


In [ ]:
# adding sentiment scores for each reviews
df['compound']=df['REVIEW_TEXT'].apply(lambda x: sentimental_score(x))
df.head()


In [ ]:
# Using SVM classifier on the train set
from sklearn.svm import LinearSVC
#classifer = LinearSVC(max_iter=df.shape[0])

In [ ]:
tfidf_lr_pipe = Pipeline([('tfidf', TfidfVectorizer(lowercase=False)), ('svm', LinearSVC(max_iter=df.shape[0]))])

In [ ]:
df['REVIEW_TEXT']=df['REVIEW_TEXT'].apply(str)
df['compound']=df['compound'].apply(str)

In [ ]:
x = df.iloc[:,1].values
x[0]

In [ ]:
y = df.iloc[:,2].values
y[0]

In [ ]:
# Splitting the train and test data (80/20)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [ ]:
# Converting the reviews to a matrix of token counts (Bag of words model)
tfidf_lr_pipe.fit(x_train, y_train)

In [ ]:
tfidf_lr_pipe.fit(x_train, y_train)

In [ ]:
y_pred = tfidf_lr_pipe.predict(x_test)

In [ ]:
predicted_df = pd.DataFrame({'predict': y_pred, 'actual': y_test})

In [ ]:
predicted_df.head(100)


In [ ]:
# Confusion matrix to see the number of true positive, true negative, false positive and false negative
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import classification_report

print('SVM results')
target_names = ['pos','neg']
print(classification_report(y_test, y_pred, target_names=target_names))

# Accuracy = (TP + TN) / (TP + TN + FP + FN)
# Precision = (TP) / (TP + FP)
# Recall = (TP) / (TP + FN)
# F1 Score = (2 * Precision * Recall) / (Precision + Recall)

In [ ]:
import pickle
filename = "../ML models/serialize_sentiment_model.pkl"  

with open(filename, 'wb') as file:  
    pickle.dump(tfidf_lr_pipe, file)